<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
 شهر‌های پرجمعیت
</font>
</h1>



In [279]:
import numpy as np
import pandas as pd

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مجموعه داده
</font>
</h2>


In [281]:
# load data
df = pd.read_excel("worldcities.xlsx")
df.head()

,ville,ville_ascii,lat,lng,pays,iso2,iso3,admin_nom,capital,population,id
0,A Coruña,A Coruna,43.3667,-8.3833,Spain,ES,ESP,Galicia,minor,245468.0,1.724417e+09
1,A Yun Pa,A Yun Pa,13.3939,108.4408,Vietnam,VN,VNM,Gia Lai,minor,53720.0,1.704946e+09
2,Aabenraa,Aabenraa,55.0444,9.4181,Denmark,DK,DNK,Syddanmark,minor,16401.0,1.208000e+09
3,Aachen,Aachen,50.7756,6.0836,Germany,DE,DEU,North Rhine-Westphalia,minor,249070.0,1.276806e+09
4,Aadorf,Aadorf,47.4939,8.8975,Switzerland,CH,CHE,Thurgau,NaN,9036.0,1.756023e+09


In [282]:
df_main = pd.read_excel("worldcities.xlsx")
df_main.shape

(44667, 11)

In [283]:
df_main.rename(columns={"pays":"country"}, inplace=True)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
حذف و نام‌گذاری ستون‌ها
</font>
</h2>


In [284]:
# drop and rename
df = df = df.drop(["ville_ascii", "capital", "id", "admin_nom"], axis=1)
df.rename(columns={"ville":"city", "pays":"country"}, inplace=True)
df.head()

,city,lat,lng,country,iso2,iso3,population
0,A Coruña,43.3667,-8.3833,Spain,ES,ESP,245468.0
1,A Yun Pa,13.3939,108.4408,Vietnam,VN,VNM,53720.0
2,Aabenraa,55.0444,9.4181,Denmark,DK,DNK,16401.0
3,Aachen,50.7756,6.0836,Germany,DE,DEU,249070.0
4,Aadorf,47.4939,8.8975,Switzerland,CH,CHE,9036.0


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
فیلتر داده
</font>
</h2>


In [285]:
# filter data
df = df = df[df["population"] >= 1000000]
df.shape

(776, 7)

In [286]:
# change type population
df["population"] = df["population"].astype('int32')
df.dtypes

city           object
lat           float64
lng           float64
country        object
iso2           object
iso3           object
population      int32
dtype: object

In [290]:
first_df = df.copy()

In [320]:
df = first_df.copy()

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
حذف داده‌های تکراری و گم‌شده
</font>
</h2>



In [331]:
# remove duplicated and missed values
df =  df.drop_duplicates()
nan_row = df.isna().sum(axis=1)
df = df[nan_row < 2]
df.shape

(767, 7)

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
پر کردن داده‌ها گم‌شده
</font>
</h2>


In [330]:
first_df.info()
df_main.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 776 entries, 19 to 44629
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        776 non-null    object 
 1   lat         774 non-null    float64
 2   lng         775 non-null    float64
 3   country     774 non-null    object 
 4   iso2        774 non-null    object 
 5   iso3        774 non-null    object 
 6   population  776 non-null    int32  
dtypes: float64(2), int32(1), object(4)
memory usage: 45.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44667 entries, 0 to 44666
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ville        44667 non-null  object 
 1   ville_ascii  44667 non-null  object 
 2   lat          44665 non-null  float64
 3   lng          44665 non-null  float64
 4   country      44665 non-null  object 
 5   iso2         44631 non-null  object 
 6   iso3         446

In [332]:
avg_lng_lat = df.groupby("country").agg({"lng": "mean", "lat": "mean"})

In [333]:
avg_lng_lat.loc["Cameroon"]

lng    10.089467
lat     4.836200
Name: Cameroon, dtype: float64

In [334]:
df['lng'] = df.apply(lambda row: row['lng'] if pd.notna(row['lng']) else avg_lng_lat.loc[row['country'], 'lng'], axis=1)
df['lat'] = df.apply(lambda row: row['lat'] if pd.notna(row['lat']) else avg_lng_lat.loc[row['country'], 'lat'], axis=1)


<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
محاسبه فاصله از تهران
</font>
</h2>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
فرمول هاورسین:
</font>
</p>



$\lambda_1$, $\lambda_2$ : longitudes of the two points in radians (lng)

$\varphi_1$, $\varphi_2$ : the latitudes of the two points in radians (lat)

$d$ : distance between two cities


$$
r = 6371
$$
$$
a = \sin^2\left(\frac{\varphi_2-\varphi_1}{2}\right) + \cos(\varphi_1)\cos(\varphi_2)\sin^2\left(\frac{\lambda_2-\lambda_1}{2}\right)
$$
$$
d = 2r \operatorname{atan2}\left(\sqrt{a}, \sqrt{1 -a}\right)
$$



In [338]:
Teh_lat = df["lat"][df["city"] == "Tehran"].values[0]
Teh_lng = df["lng"][df["city"] == "Tehran"].values[0]
Teh_lat

35.6892

In [339]:
df["haversin_from_teh"] = (np.sin((np.deg2rad(df["lat"]) - np.deg2rad(Teh_lat)) / 2)**2)) + (
np.cos(np.deg2rad(df["lat"])) * np.cos(np.deg2rad(Teh_lat)) * np.sin(np.deg2rad((df["lng"]- Teh_lng)/2))**2)


In [340]:
# create distance_from_tehran column
df["distance_from_tehran"] = 2 * 6371 * np.arctan2(np.sqrt(df["haversin_from_teh"]), np.sqrt(1-df["haversin_from_teh"]))

<h2 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
مرتب‌سازی و دخیره دیتافریم
</font>
</h2>


In [343]:
# sort and save dataframe df 
df.sort_values(by=["city", "lat"], ascending=[True, False], inplace=True)
df.to_csv("distances.csv", index=False)

In [344]:
# sort and save dataframe first_df
first_df.sort_values(by=["city", "lat"], ascending=[True, False], inplace=True)
first_df.to_csv("cities.csv", index=False)